# Lab 01: Data Collection and Preprocessing

This notebook demonstrates fundamental data collection and preprocessing techniques using Python and pandas.

---
## Step 1: Load the Dataset

In this step, we load the retail transactions CSV file using pandas and inspect the first few rows to understand the data structure.

In [ ]:
# Import pandas library
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('data/Retail_Transactions_Dataset.csv')

# Display the first 3 rows
df.head(3)

---
## Step 2: Data Structure Choice

### Why pandas DataFrame?

We use a **pandas DataFrame** to store our retail transactions data for the following reasons:

| Criteria | DataFrame Advantage |
|----------|---------------------|
| **Tabular format** | CSV data is naturally row-column structured; DataFrames mirror this exactly |
| **Mixed data types** | Our dataset contains strings (`Product`, `City`), numbers (`Total_Cost`, `Total_Items`), and dates (`Date`) — DataFrames handle heterogeneous columns efficiently |
| **Built-in methods** | Pandas provides optimized functions for filtering, grouping, aggregating, and cleaning data |
| **Memory efficiency** | DataFrames use NumPy arrays under the hood, enabling vectorized operations |
| **Ecosystem integration** | Seamless compatibility with visualization libraries (matplotlib, seaborn) and ML frameworks (scikit-learn) |

### Alternatives Considered

- **Python lists/dicts**: No built-in support for column operations or missing data handling
- **NumPy arrays**: Require homogeneous data types; not ideal for mixed-type tabular data
- **SQL database**: Overkill for a single-file analysis; adds setup complexity

**Conclusion**: pandas DataFrame is the optimal choice for exploratory data analysis and preprocessing of structured CSV data.

---
## Step 3: Transaction Class

We define a `Transaction` class to encapsulate individual transaction records. This provides:
- **Encapsulation**: Each transaction is a self-contained object
- **Data cleaning**: The `clean()` method standardizes and sanitizes field values
- **Calculations**: The `total()` method computes the transaction total

In [ ]:
class Transaction:
    """
    Represents a single retail transaction.
    
    Attributes:
        transaction_id: Unique identifier for the transaction
        date: Date of the transaction
        customer_name: Name of the customer
        product: Product purchased
        quantity: Number of items (Total_Items)
        price: Cost of the transaction (Total_Cost)
        city: City where the transaction occurred
        promotion: Promotion/coupon code applied
    """
    
    def __init__(self, row):
        """Initialize a Transaction from a DataFrame row."""
        self.transaction_id = row.get('Transaction_ID', None)
        self.date = row.get('Date', None)
        self.customer_name = row.get('Customer_Name', None)
        self.product = row.get('Product', None)
        self.quantity = row.get('Total_Items', 0)
        self.price = row.get('Total_Cost', 0.0)
        self.city = row.get('City', None)
        self.promotion = row.get('Promotion', None)
    
    def clean(self):
        """
        Clean and standardize transaction data.
        
        Returns:
            self: The Transaction instance (for method chaining)
        """
        # Strip whitespace from string fields
        if isinstance(self.customer_name, str):
            self.customer_name = self.customer_name.strip().title()
        
        if isinstance(self.product, str):
            self.product = self.product.strip().title()
        
        if isinstance(self.city, str):
            self.city = self.city.strip().title()
        
        if isinstance(self.promotion, str):
            self.promotion = self.promotion.strip().upper()
        
        # Handle missing/invalid numeric values
        try:
            self.quantity = int(self.quantity) if self.quantity else 0
        except (ValueError, TypeError):
            self.quantity = 0
        
        try:
            self.price = float(self.price) if self.price else 0.0
        except (ValueError, TypeError):
            self.price = 0.0
        
        return self
    
    def total(self):
        """
        Calculate the transaction total.
        
        Returns:
            float: The total cost of the transaction
        """
        return float(self.price)
    
    def __repr__(self):
        """String representation for debugging."""
        return f"Transaction(id={self.transaction_id}, product={self.product}, total={self.total():.2f})"


# --- Demo: Create and clean a Transaction ---
# Get the first row as a dictionary
sample_row = df.iloc[0].to_dict()

# Create a Transaction object
txn = Transaction(sample_row)

# Clean the transaction data
txn.clean()

# Display the transaction
print(f"Transaction ID: {txn.transaction_id}")
print(f"Customer:       {txn.customer_name}")
print(f"Product:        {txn.product}")
print(f"Quantity:       {txn.quantity}")
print(f"City:           {txn.city}")
print(f"Promotion:      {txn.promotion}")
print(f"Total:          ${txn.total():.2f}")

---
## Step 4: Bulk Load Transactions

Convert the entire DataFrame into a list of `Transaction` objects. This demonstrates:
- Iterating over DataFrame rows
- Creating objects in bulk
- Applying the `clean()` method to each transaction

In [ ]:
# Convert DataFrame rows to a list of Transaction objects
# Each row becomes a Transaction, then we call clean() on it

transactions = []

for index, row in df.iterrows():
    # Convert row to dictionary and create Transaction
    txn = Transaction(row.to_dict())
    
    # Clean the transaction data
    txn.clean()
    
    # Add to list
    transactions.append(txn)

# Display summary
print(f"Total transactions loaded: {len(transactions):,}")
print(f"\nFirst 3 transactions:")
for t in transactions[:3]:
    print(f"  {t}")

print(f"\nLast 3 transactions:")
for t in transactions[-3:]:
    print(f"  {t}")

---
## Step 5: Data Profiling

Compute basic statistics to understand the dataset:
- **Price statistics**: min, mean, max
- **Unique cities**: count of distinct shipping locations

In [ ]:
# Extract prices from all transactions
prices = [t.price for t in transactions]

# Compute price statistics
min_price = min(prices)
max_price = max(prices)
mean_price = sum(prices) / len(prices)

# Get unique cities
cities = set(t.city for t in transactions)
unique_city_count = len(cities)

# Display profiling results
print("=" * 40)
print("DATA PROFILING RESULTS")
print("=" * 40)

print(f"\n📊 Price Statistics:")
print(f"   Min:  ${min_price:,.2f}")
print(f"   Mean: ${mean_price:,.2f}")
print(f"   Max:  ${max_price:,.2f}")

print(f"\n🏙️ City Statistics:")
print(f"   Unique cities: {unique_city_count}")

print(f"\n📋 Sample cities:")
for city in sorted(cities)[:5]:
    print(f"   - {city}")

---
## Step 6: Identify Dirty Data

Before cleaning, we analyze the raw DataFrame to identify data quality issues:
- **Missing values**: Null or NaN entries
- **Duplicates**: Repeated transaction IDs
- **Invalid values**: Negative prices, zero quantities
- **Inconsistent formatting**: Mixed case, extra whitespace

In [ ]:
# ============================================
# DIRTY DATA IDENTIFICATION
# ============================================

print("=" * 50)
print("DIRTY DATA REPORT")
print("=" * 50)

# --- 1. Missing Values ---
print("\n1️⃣ MISSING VALUES")
print("-" * 30)
missing_counts = df.isnull().sum()
total_missing = missing_counts.sum()

if total_missing > 0:
    for col, count in missing_counts.items():
        if count > 0:
            pct = (count / len(df)) * 100
            print(f"   {col}: {count:,} ({pct:.2f}%)")
else:
    print("   No missing values found")

# --- 2. Duplicate Transaction IDs ---
print("\n2️⃣ DUPLICATE TRANSACTION IDs")
print("-" * 30)
duplicate_ids = df[df.duplicated(subset=['Transaction_ID'], keep=False)]
dup_count = len(duplicate_ids)

if dup_count > 0:
    print(f"   Count: {dup_count:,} rows with duplicate IDs")
    print(f"   Examples:")
    dup_examples = duplicate_ids['Transaction_ID'].value_counts().head(3)
    for tid, cnt in dup_examples.items():
        print(f"      ID {tid}: appears {cnt} times")
else:
    print("   No duplicate Transaction IDs found")

# --- 3. Invalid Prices ---
print("\n3️⃣ INVALID PRICES (negative or zero)")
print("-" * 30)
invalid_prices = df[df['Total_Cost'] <= 0]
inv_price_count = len(invalid_prices)

if inv_price_count > 0:
    print(f"   Count: {inv_price_count:,}")
    print(f"   Examples:")
    for idx, row in invalid_prices.head(3).iterrows():
        print(f"      Row {idx}: ${row['Total_Cost']}")
else:
    print("   No invalid prices found")

# --- 4. Invalid Quantities ---
print("\n4️⃣ INVALID QUANTITIES (zero or negative)")
print("-" * 30)
invalid_qty = df[df['Total_Items'] <= 0]
inv_qty_count = len(invalid_qty)

if inv_qty_count > 0:
    print(f"   Count: {inv_qty_count:,}")
    print(f"   Examples:")
    for idx, row in invalid_qty.head(3).iterrows():
        print(f"      Row {idx}: {row['Total_Items']} items")
else:
    print("   No invalid quantities found")

# --- 5. Whitespace Issues ---
print("\n5️⃣ WHITESPACE ISSUES (leading/trailing spaces)")
print("-" * 30)
string_cols = ['Customer_Name', 'Product', 'City', 'Promotion']
whitespace_issues = 0

for col in string_cols:
    if col in df.columns:
        # Check for leading/trailing whitespace
        has_whitespace = df[col].astype(str).str.contains(r'^\s+|\s+$', regex=True, na=False)
        count = has_whitespace.sum()
        if count > 0:
            whitespace_issues += count
            example = df.loc[has_whitespace, col].iloc[0] if count > 0 else "N/A"
            print(f"   {col}: {count:,} rows")
            print(f"      Example: '{example}'")

if whitespace_issues == 0:
    print("   No whitespace issues found")

# --- Summary ---
print("\n" + "=" * 50)
print("SUMMARY")
print("=" * 50)
print(f"   Total rows:              {len(df):,}")
print(f"   Missing values:          {total_missing:,}")
print(f"   Duplicate IDs:           {dup_count:,}")
print(f"   Invalid prices:          {inv_price_count:,}")
print(f"   Invalid quantities:      {inv_qty_count:,}")
print(f"   Whitespace issues:       {whitespace_issues:,}")

---
## Step 7: Cleaning Rules

The `clean()` method applies the following transformations:

| Rule | Before | After |
|------|--------|-------|
| Strip whitespace | `"  John  "` | `"John"` |
| Title case names | `"john DOE"` | `"John Doe"` |
| Uppercase promos | `"save20"` | `"SAVE20"` |
| Handle null strings | `None` | `None` (unchanged) |
| Convert quantities | `"5"` or `NaN` | `5` or `0` |
| Convert prices | `"99.99"` or `NaN` | `99.99` or `0.0` |

Below we demonstrate before/after comparisons on sample data.

In [ ]:
# ============================================
# BEFORE/AFTER CLEANING COMPARISON
# ============================================

# Create sample dirty data to demonstrate cleaning
dirty_samples = [
    {
        'Transaction_ID': 'TEST001',
        'Date': '2024-01-15',
        'Customer_Name': '  john DOE  ',      # whitespace + wrong case
        'Product': 'LAPTOP computer',          # inconsistent case
        'Total_Items': '3',                    # string instead of int
        'Total_Cost': '999.99',                # string instead of float
        'City': '  new york  ',                # whitespace + lowercase
        'Promotion': 'save20'                  # lowercase promo code
    },
    {
        'Transaction_ID': 'TEST002',
        'Date': '2024-01-16',
        'Customer_Name': 'JANE SMITH',         # all caps
        'Product': '  wireless mouse  ',       # whitespace
        'Total_Items': None,                   # missing value
        'Total_Cost': None,                    # missing value
        'City': 'LOS ANGELES',                 # all caps
        'Promotion': '  PROMO50  '             # whitespace
    },
    {
        'Transaction_ID': 'TEST003',
        'Date': '2024-01-17',
        'Customer_Name': 'bob wilson',         # all lowercase
        'Product': 'USB Cable',                # mixed case (correct)
        'Total_Items': 0,                      # zero quantity
        'Total_Cost': -50.00,                  # negative price
        'City': 'chicago',                     # lowercase
        'Promotion': None                      # no promo
    }
]

print("=" * 70)
print("BEFORE/AFTER CLEANING COMPARISON")
print("=" * 70)

for i, dirty_row in enumerate(dirty_samples, 1):
    print(f"\n{'─' * 70}")
    print(f"SAMPLE {i}")
    print(f"{'─' * 70}")
    
    # Create transaction WITHOUT cleaning
    before = Transaction(dirty_row)
    
    # Create transaction WITH cleaning
    after = Transaction(dirty_row)
    after.clean()
    
    # Display comparison
    print(f"\n{'Field':<15} {'BEFORE':<25} {'AFTER':<25}")
    print(f"{'-' * 15} {'-' * 25} {'-' * 25}")
    
    print(f"{'customer_name':<15} {repr(before.customer_name):<25} {repr(after.customer_name):<25}")
    print(f"{'product':<15} {repr(before.product):<25} {repr(after.product):<25}")
    print(f"{'city':<15} {repr(before.city):<25} {repr(after.city):<25}")
    print(f"{'promotion':<15} {repr(before.promotion):<25} {repr(after.promotion):<25}")
    print(f"{'quantity':<15} {repr(before.quantity):<25} {repr(after.quantity):<25}")
    print(f"{'price':<15} {repr(before.price):<25} {repr(after.price):<25}")

print(f"\n{'=' * 70}")
print("CLEANING RULES APPLIED:")
print("=" * 70)
print("  1. strip()    - Remove leading/trailing whitespace")
print("  2. title()    - Convert to Title Case (names, products, cities)")
print("  3. upper()    - Convert to UPPERCASE (promotion codes)")
print("  4. int()      - Convert quantity to integer, default to 0")
print("  5. float()    - Convert price to float, default to 0.0")

---
## Step 8: Transform Coupon Codes

Extract discount values from promotion codes using regular expressions:

| Promotion Code | Extracted Discount |
|----------------|-------------------|
| `SAVE20` | `20` |
| `PROMO50` | `50` |
| `DISCOUNT15OFF` | `15` |
| `FREESHIP` | `0` (no numeric value) |
| `None` | `0` |

**Regex pattern**: `\d+` matches one or more digits in the promotion string.

In [ ]:
import re

def extract_discount(promotion):
    """
    Extract numeric discount value from a promotion code using regex.
    
    Args:
        promotion: Promotion code string (e.g., 'SAVE20', 'PROMO50')
    
    Returns:
        int: Extracted discount value, or 0 if no number found
    """
    if promotion is None:
        return 0
    
    # Convert to string in case of non-string input
    promo_str = str(promotion)
    
    # Regex pattern: match one or more digits
    match = re.search(r'\d+', promo_str)
    
    if match:
        return int(match.group())
    return 0


# ============================================
# DEMONSTRATE DISCOUNT EXTRACTION
# ============================================

print("=" * 60)
print("COUPON CODE TRANSFORMATION")
print("=" * 60)

# Test cases
test_promotions = [
    'SAVE20',
    'PROMO50',
    'DISCOUNT15OFF',
    'GET10PCT',
    'FREESHIP',
    'SUMMER2024SALE25',
    None,
    ''
]

print(f"\n{'Promotion Code':<25} {'Extracted Discount':<20}")
print(f"{'-' * 25} {'-' * 20}")

for promo in test_promotions:
    discount = extract_discount(promo)
    promo_display = repr(promo) if promo is not None else 'None'
    print(f"{promo_display:<25} {discount}%")

# ============================================
# APPLY TO ALL TRANSACTIONS
# ============================================

print(f"\n{'=' * 60}")
print("APPLYING TO DATASET")
print("=" * 60)

# Extract discounts for all transactions
discounts = [extract_discount(t.promotion) for t in transactions]

# Count transactions with discounts
with_discount = sum(1 for d in discounts if d > 0)
without_discount = len(discounts) - with_discount

print(f"\n📊 Discount Statistics:")
print(f"   Transactions with discount:    {with_discount:,}")
print(f"   Transactions without discount: {without_discount:,}")

# Show discount distribution
discount_counts = {}
for d in discounts:
    discount_counts[d] = discount_counts.get(d, 0) + 1

print(f"\n📈 Discount Distribution (top 5):")
for discount, count in sorted(discount_counts.items(), key=lambda x: -x[1])[:5]:
    pct = (count / len(discounts)) * 100
    print(f"   {discount}% discount: {count:,} transactions ({pct:.1f}%)")

# Show sample transactions with extracted discounts
print(f"\n📋 Sample Transactions with Discounts:")
print(f"{'Promotion':<20} {'Discount':<10} {'Price':<15}")
print(f"{'-' * 20} {'-' * 10} {'-' * 15}")

sample_count = 0
for t in transactions:
    discount = extract_discount(t.promotion)
    if discount > 0 and sample_count < 5:
        print(f"{str(t.promotion):<20} {discount}%{'':<8} ${t.price:,.2f}")
        sample_count += 1

---
## Step 9: Feature Engineering

Add a computed property `days_since_purchase` using Python's `datetime` module and the `@property` decorator.

**Benefits of `@property`:**
- Computed on-demand (not stored)
- Accessed like an attribute: `txn.days_since_purchase`
- Encapsulates date parsing logic
- Always returns current value relative to today

In [ ]:
from datetime import datetime, date

class TransactionWithFeatures(Transaction):
    """
    Extended Transaction class with feature engineering.
    
    Inherits from Transaction and adds computed properties.
    """
    
    @property
    def days_since_purchase(self):
        """
        Calculate the number of days since the purchase date.
        
        Returns:
            int: Days between purchase date and today, or -1 if date is invalid
        """
        if self.date is None:
            return -1
        
        try:
            # Parse the date string (handles multiple formats)
            if isinstance(self.date, str):
                # Try common date formats
                for fmt in ['%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y', '%Y/%m/%d']:
                    try:
                        purchase_date = datetime.strptime(self.date, fmt).date()
                        break
                    except ValueError:
                        continue
                else:
                    # No format matched
                    return -1
            elif isinstance(self.date, (datetime, date)):
                purchase_date = self.date if isinstance(self.date, date) else self.date.date()
            else:
                return -1
            
            # Calculate difference from today
            today = date.today()
            delta = today - purchase_date
            return delta.days
            
        except (ValueError, TypeError):
            return -1
    
    @property
    def discount_amount(self):
        """
        Calculate the discount amount based on promotion code.
        
        Returns:
            float: Discount amount in dollars
        """
        discount_pct = extract_discount(self.promotion)
        return (discount_pct / 100) * self.price


# ============================================
# DEMONSTRATE FEATURE ENGINEERING
# ============================================

print("=" * 60)
print("FEATURE ENGINEERING: days_since_purchase")
print("=" * 60)

# Test with sample dates
test_rows = [
    {'Transaction_ID': 'T001', 'Date': '2024-01-15', 'Product': 'Laptop', 
     'Total_Cost': 999.99, 'Total_Items': 1, 'Promotion': 'SAVE10'},
    {'Transaction_ID': 'T002', 'Date': '2025-01-01', 'Product': 'Phone', 
     'Total_Cost': 599.99, 'Total_Items': 1, 'Promotion': 'PROMO20'},
    {'Transaction_ID': 'T003', 'Date': '2025-12-25', 'Product': 'Tablet', 
     'Total_Cost': 399.99, 'Total_Items': 2, 'Promotion': None},
    {'Transaction_ID': 'T004', 'Date': None, 'Product': 'Cable', 
     'Total_Cost': 19.99, 'Total_Items': 3, 'Promotion': 'SAVE5'},
]

print(f"\n{'Date':<15} {'Days Since':<12} {'Product':<15} {'Discount Amt':<12}")
print(f"{'-' * 15} {'-' * 12} {'-' * 15} {'-' * 12}")

for row in test_rows:
    txn = TransactionWithFeatures(row)
    txn.clean()
    days = txn.days_since_purchase
    days_str = str(days) if days >= 0 else "Invalid"
    discount_amt = txn.discount_amount
    print(f"{str(row['Date']):<15} {days_str:<12} {txn.product:<15} ${discount_amt:,.2f}")

# ============================================
# APPLY TO REAL TRANSACTIONS
# ============================================

print(f"\n{'=' * 60}")
print("APPLYING TO DATASET")
print("=" * 60)

# Convert existing transactions to TransactionWithFeatures
enhanced_transactions = []
for t in transactions[:1000]:  # Sample first 1000 for demo
    row = {
        'Transaction_ID': t.transaction_id,
        'Date': t.date,
        'Customer_Name': t.customer_name,
        'Product': t.product,
        'Total_Items': t.quantity,
        'Total_Cost': t.price,
        'City': t.city,
        'Promotion': t.promotion
    }
    enhanced = TransactionWithFeatures(row)
    enhanced.clean()
    enhanced_transactions.append(enhanced)

# Calculate statistics on days_since_purchase
valid_days = [t.days_since_purchase for t in enhanced_transactions if t.days_since_purchase >= 0]

if valid_days:
    print(f"\n📊 Days Since Purchase Statistics (sample of 1,000):")
    print(f"   Min:  {min(valid_days):,} days")
    print(f"   Max:  {max(valid_days):,} days")
    print(f"   Mean: {sum(valid_days) / len(valid_days):,.1f} days")
    print(f"   Valid dates: {len(valid_days):,}")
    print(f"   Invalid dates: {len(enhanced_transactions) - len(valid_days):,}")

# Show sample enhanced transactions
print(f"\n📋 Sample Enhanced Transactions:")
print(f"{'ID':<10} {'Date':<12} {'Days Ago':<10} {'Product':<20}")
print(f"{'-' * 10} {'-' * 12} {'-' * 10} {'-' * 20}")

for t in enhanced_transactions[:5]:
    days = t.days_since_purchase
    days_str = str(days) if days >= 0 else "N/A"
    print(f"{str(t.transaction_id):<10} {str(t.date):<12} {days_str:<10} {str(t.product)[:20]:<20}")

---
## Step 10: Aggregation

Aggregate total revenue by shipping city to identify top-performing markets.

**Aggregation approach:**
1. Group transactions by `city`
2. Sum `price` (revenue) for each city
3. Count transactions per city
4. Sort by revenue descending

In [ ]:
# ============================================
# REVENUE AGGREGATION BY CITY
# ============================================

def aggregate_revenue_by_city(transactions):
    """
    Aggregate total revenue and transaction count by shipping city.
    
    Args:
        transactions: List of Transaction objects
    
    Returns:
        dict: {city: {'revenue': float, 'count': int, 'avg_order': float}}
    """
    city_stats = {}
    
    for t in transactions:
        city = t.city if t.city else 'Unknown'
        
        if city not in city_stats:
            city_stats[city] = {'revenue': 0.0, 'count': 0}
        
        city_stats[city]['revenue'] += t.price
        city_stats[city]['count'] += 1
    
    # Calculate average order value for each city
    for city in city_stats:
        count = city_stats[city]['count']
        revenue = city_stats[city]['revenue']
        city_stats[city]['avg_order'] = revenue / count if count > 0 else 0
    
    return city_stats


# Aggregate revenue by city
city_revenue = aggregate_revenue_by_city(transactions)

# Sort by revenue (descending)
sorted_cities = sorted(city_revenue.items(), key=lambda x: x[1]['revenue'], reverse=True)

# Calculate totals
total_revenue = sum(stats['revenue'] for _, stats in sorted_cities)
total_transactions = sum(stats['count'] for _, stats in sorted_cities)

print("=" * 70)
print("REVENUE AGGREGATION BY SHIPPING CITY")
print("=" * 70)

print(f"\n📊 Overall Statistics:")
print(f"   Total Revenue:      ${total_revenue:,.2f}")
print(f"   Total Transactions: {total_transactions:,}")
print(f"   Unique Cities:      {len(city_revenue):,}")

# Top 10 cities by revenue
print(f"\n🏆 TOP 10 CITIES BY REVENUE")
print(f"{'-' * 70}")
print(f"{'Rank':<6} {'City':<20} {'Revenue':<18} {'Transactions':<14} {'Avg Order':<12}")
print(f"{'-' * 6} {'-' * 20} {'-' * 18} {'-' * 14} {'-' * 12}")

for rank, (city, stats) in enumerate(sorted_cities[:10], 1):
    revenue = stats['revenue']
    count = stats['count']
    avg = stats['avg_order']
    pct = (revenue / total_revenue) * 100
    print(f"{rank:<6} {city[:20]:<20} ${revenue:>14,.2f} {count:>14,} ${avg:>10,.2f}")

# Bottom 5 cities by revenue
print(f"\n📉 BOTTOM 5 CITIES BY REVENUE")
print(f"{'-' * 70}")
print(f"{'Rank':<6} {'City':<20} {'Revenue':<18} {'Transactions':<14} {'Avg Order':<12}")
print(f"{'-' * 6} {'-' * 20} {'-' * 18} {'-' * 14} {'-' * 12}")

for rank, (city, stats) in enumerate(sorted_cities[-5:], len(sorted_cities) - 4):
    revenue = stats['revenue']
    count = stats['count']
    avg = stats['avg_order']
    print(f"{rank:<6} {city[:20]:<20} ${revenue:>14,.2f} {count:>14,} ${avg:>10,.2f}")

# Revenue distribution summary
print(f"\n📈 REVENUE DISTRIBUTION")
print(f"{'-' * 70}")

# Calculate concentration
top_5_revenue = sum(stats['revenue'] for _, stats in sorted_cities[:5])
top_10_revenue = sum(stats['revenue'] for _, stats in sorted_cities[:10])

print(f"   Top 5 cities:  ${top_5_revenue:>14,.2f} ({(top_5_revenue/total_revenue)*100:.1f}% of total)")
print(f"   Top 10 cities: ${top_10_revenue:>14,.2f} ({(top_10_revenue/total_revenue)*100:.1f}% of total)")
print(f"   Remaining:     ${total_revenue - top_10_revenue:>14,.2f} ({((total_revenue - top_10_revenue)/total_revenue)*100:.1f}% of total)")

---
## Step 11: Serialization

Save the cleaned transaction data to multiple formats for downstream use:

| Format | File | Use Case |
|--------|------|----------|
| **CSV** | `cleaned_transactions.csv` | Spreadsheet analysis, SQL import |
| **JSON** | `cleaned_transactions.json` | Web APIs, NoSQL databases |

In [ ]:
import json
import csv

# ============================================
# SAVE CLEANED DATA TO CSV AND JSON
# ============================================

def transaction_to_dict(t):
    """Convert a Transaction object to a dictionary."""
    return {
        'transaction_id': t.transaction_id,
        'date': t.date,
        'customer_name': t.customer_name,
        'product': t.product,
        'quantity': t.quantity,
        'price': t.price,
        'city': t.city,
        'promotion': t.promotion,
        'discount_pct': extract_discount(t.promotion)
    }

# Convert all transactions to dictionaries
cleaned_data = [transaction_to_dict(t) for t in transactions]

# --- Save to CSV ---
csv_path = 'data/cleaned_transactions.csv'
fieldnames = ['transaction_id', 'date', 'customer_name', 'product', 
              'quantity', 'price', 'city', 'promotion', 'discount_pct']

with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(cleaned_data)

# --- Save to JSON ---
json_path = 'data/cleaned_transactions.json'

with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(cleaned_data, f, indent=2, ensure_ascii=False)

# --- Verify files ---
import os

csv_size = os.path.getsize(csv_path)
json_size = os.path.getsize(json_path)

print("=" * 60)
print("SERIALIZATION COMPLETE")
print("=" * 60)

print(f"\n📁 Files Created:")
print(f"   CSV:  {csv_path}")
print(f"         Size: {csv_size:,} bytes ({csv_size/1024/1024:.2f} MB)")
print(f"   JSON: {json_path}")
print(f"         Size: {json_size:,} bytes ({json_size/1024/1024:.2f} MB)")

print(f"\n📊 Records Written: {len(cleaned_data):,}")

# Preview CSV content
print(f"\n📋 CSV Preview (first 3 rows):")
print(f"{'-' * 60}")
with open(csv_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 4:  # header + 3 rows
            print(line.strip()[:80] + "..." if len(line) > 80 else line.strip())

# Preview JSON content
print(f"\n📋 JSON Preview (first record):")
print(f"{'-' * 60}")
print(json.dumps(cleaned_data[0], indent=2))

---
## Step 12: Reflection

### Functions vs Methods

In this lab, we used both **functions** and **methods** to process data:

| Type | Example | Key Difference |
|------|---------|----------------|
| **Function** | `extract_discount(promo)` | Standalone, takes data as arguments |
| **Method** | `txn.clean()` | Bound to an object, operates on `self` |

**Why this matters:** Methods encapsulate behavior with data—the `Transaction` class bundles attributes (`price`, `city`) with operations (`clean()`, `total()`). This promotes reusability and maintainability. Functions like `aggregate_revenue_by_city()` remain independent when they operate on collections rather than single objects.

**Key takeaway:** Use methods for object-specific behavior; use functions for general-purpose transformations.

---
## Data Dictionary

### Primary Dataset: `Retail_Transactions_Dataset.csv`

| Column | Data Type | Description | Example |
|--------|-----------|-------------|---------|
| `Transaction_ID` | string | Unique identifier for each transaction | `TXN001` |
| `Date` | string | Date of purchase (YYYY-MM-DD) | `2024-01-15` |
| `Customer_Name` | string | Full name of the customer | `John Doe` |
| `Product` | string | Name of the product purchased | `Laptop` |
| `Total_Items` | integer | Quantity of items purchased | `3` |
| `Total_Cost` | float | Total transaction amount in USD | `299.99` |
| `Payment_Method` | string | Method of payment used | `Credit Card` |
| `City` | string | Shipping/billing city | `New York` |
| `Store_Type` | string | Type of store (online/physical) | `Online` |
| `Discount_Applied` | boolean | Whether a discount was applied | `True` |
| `Customer_Category` | string | Customer segment classification | `Premium` |
| `Season` | string | Season of purchase | `Winter` |
| `Promotion` | string | Promotion/coupon code used | `SAVE20` |

---

### Secondary Dataset: `cleaned_transactions.csv` / `.json`

| Column | Data Type | Description | Transformation Applied |
|--------|-----------|-------------|------------------------|
| `transaction_id` | string | Unique transaction identifier | Renamed from `Transaction_ID` |
| `date` | string | Purchase date | No change |
| `customer_name` | string | Customer name (cleaned) | `strip().title()` applied |
| `product` | string | Product name (cleaned) | `strip().title()` applied |
| `quantity` | integer | Items purchased | Converted to int, default 0 |
| `price` | float | Transaction total | Converted to float, default 0.0 |
| `city` | string | City (cleaned) | `strip().title()` applied |
| `promotion` | string | Promo code (cleaned) | `strip().upper()` applied |
| `discount_pct` | integer | **Derived field** - extracted discount % | Regex extraction from promotion |

---

### Derived Features (TransactionWithFeatures class)

| Property | Data Type | Description | Computation |
|----------|-----------|-------------|-------------|
| `days_since_purchase` | integer | Days between purchase and today | `date.today() - purchase_date` |
| `discount_amount` | float | Discount in dollars | `(discount_pct / 100) * price` |